## Train model
Danh sách class: <br>
    0. Correct <br>
    1. Chân quá hẹp <br >
    2. Chân quá rộng <br >
    3. Khoảng cách giữa 2 đầu gối quá nhỏ <br >
    4. Xuống quá xâu <br >
    5. Lỗi gập gập lưng <br >

In [ ]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Khởi tạo keypoints và các phương thức 
Các keypoints cần lấy :
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",

Dataframe gồm label và các features là toạ độ keypoints gồm 4 thành phần: x, y, z, visibility
- x, y: tọa độ x, y của keypoints đã được chuẩn hóa min max scale
- z: độ sâu của điểm ảnh theo điểm giữa hông 
- visibility: biểu thị khả năng có thể nhìn thấy hoặc che khuất
 

In [ ]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header.append("video_name")
header

In [ ]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """

    #Độ sâu ảnh
    t = 33;
    A = np.array([pointA[0], pointA[1], pointA[2]/2])  # (xA, yA, zA)
    B = np.array([pointB[0], pointB[1], pointB[2]/2 ])  # (xB, yB, zB)
    C = np.array([pointC[0], pointC[1], pointC[2]/2])  # (xC, yC, zC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle

#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


In [ ]:
#Lỗi 1 - 2: ratio = feet/shoulder
Min_radio_feet_shoulder = 0.59
Max_radio_feet_shoulder = 1

#Lỗi 3 : ratio = knee/feet
Min_up_radio_knee_feet = 0.85 
Min_middle_ratio_knee_feet = 1.04
Min_down_ratio_knee_feet = 1.46

#Lỗi 4: knee angle < 50
Min_knee_angle = 50.97 

#Lỗi 5: ratio = knee_angle/hip_angle
Max_ratio_knee_hip = 1


In [ ]:
df = pd.DataFrame(columns=header)
df_test = pd.DataFrame(columns=header)
df_va = pd.DataFrame(columns=header)
df 

# 2. Hàm duyệt data lỗi chân quá hẹp

In [ ]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_1_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân (sửa từ foot_index thành ankle)
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)


                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width < Min_radio_feet_shoulder):
                    row.append(1)
                # else:
                #     row.append(0)
                
                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                row.extend([Video_name])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [ ]:
import os

video_folder = "Data/Train/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "train")

video_folder = "Data/Test/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "validate")


In [ ]:
df 

# 2. Lỗi chân rộng


In [ ]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_2_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân
                left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
                right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

                feet_width = calculate_distance(left_foot_index, right_foot_index)

                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width > Max_radio_feet_shoulder):
                    row.append(2)
                # else:
                #     row.append(0)

                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                row.extend([Video_name])
                        
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [ ]:
import os

video_folder = "Data/Train/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "train")

video_folder = "Data/Test/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_2_error(video_folder, video_file, "validate")

In [ ]:
df

# 3. Lỗi gối hẹp


In [ ]:
#import model để dự đoán giai đoạn
import pickle


with open("LR_Up_Down_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

In [ ]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_3_error(Video_folder = "Data/train/Dau_goi_hep" , Video_name = "20250228_092009000_iOS.mp4" , type = "train"):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính khoảng cách
            knee_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:-1])

                # Tinhs toán và so sánh khoảng cách đầu gối, chân
                landmarks = results.pose_landmarks.landmark

                # Lấy trạng thái hiện tại 
                predicted_class = trained_model.predict(X)[0]
                prediction_probability = trained_model.predict_proba(X)[0]

                current_stage ="middle"
                if prediction_probability[prediction_probability.argmax()] >= 0.7: # Nếu xác suất dự đoán lớn hơn 0.5 thì mới cập nhật trạng thái
                    if predicted_class == 0:
                        current_stage = "down"
                    elif current_stage == "middle" and predicted_class == 1:
                        current_stage = "up"
                else:
                    current_stage = "middle"

                # print(f"Predicted class: {predicted_class}, Probability: {prediction_probability}, Current stage: {current_stage}")
            
                # Khoảng cách giữa 2 đầu gối 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                knee_width = calculate_distance(left_knee, right_knee)

                # Khoảng cách giữa 2 chân
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                feet_width = calculate_distance(left_ankle, right_ankle)

                ratio = knee_width/feet_width
                # print(f"Knee width: {knee_width}, Feet width: {feet_width}, Ratio: {ratio}")
                # print("\n ------------------- \n")

                # Kiểm tra lỗi
                row = []
                isError = False
                if(current_stage == "down" and ratio < Min_down_ratio_knee_feet):
                    isError = True
                elif(current_stage == "middle" and ratio < Min_middle_ratio_knee_feet):
                    isError = True


                if isError:
                    row.append(3)
                # else :
                #     row.append(0)
                
                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                row.extend([Video_name])
                # Kiểm tra số lượng phần tử trong row và số lượng cột trong df
                print(f"Số lượng phần tử trong row: {len(row)}")
                print(f"Số lượng cột trong df: {len(df.columns)}")

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")


                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee distance
                cv2.putText(image, "KNEE", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display current stage
                cv2.putText(image, "STAGE", (300, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (300, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị


        Cap.release()

detection_3_error()

In [ ]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/Train/Dau_goi_hep"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "train")

video_folder = "Data/Test/Dau_goi_hep"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Dau_goi_hep"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_3_error(video_folder, video_file, "validate")

In [ ]:
df 

# 4.Lỗi xuống quá xâu

In [ ]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def detection_4_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None

            
            try:

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                row = []
                if(knee_angle < Min_knee_angle):
                    row.append(4)
                # else:
                #     row.append(0)
                
                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                row.extend([Video_name])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                #Display label
                cv2.putText(image, "LABEL", (10, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_angle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # if(knee_angle < 55):
            plt.imshow(image)
            plt.axis("off")  # Ẩn trục tọa độ
            plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# video_folder = "Data/Train/Gap_lung"
# video_files = "20250228_165326.mp4"

# video_folder = "Data/Train/Xuong_qua_xau"
# video_files = "20250228_163416.mp4"

# process_frame(video_folder, video_files)



In [ ]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
video_folder = "Data/Train/Xuong_qua_xau"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_4_error(video_folder, video_file, "train")

# video_folder = "Data/Test/Xuong_qua_xau"
# # video_folder = "Data/test/Correct"
# video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

# for video_file in video_files:
#     detection_4_error(video_folder, video_file, "test")

# video_folder = "Data/Validation/Xuong_qua_xau"
# # video_folder = "Data/validate/Correct"
# video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

# for video_file in video_files:
#     detection_4_error(video_folder, video_file, "validate")

In [ ]:
df

# 5. Lỗi gập lưng


In [ ]:
def detection_5_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                #Góc hông phải
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]

                right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

                #Góc hông trái
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]

                left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                #Lấy trung bình cộng 2 góc
                hip_angle = (left_hip_angle + right_hip_angle) / 2

                ratio = knee_angle/hip_angle

                #Kiểm tra lỗi
                row = []
                if(ratio > Max_ratio_knee_hip):
                    row.append(5)
                # else:
                #     row.append(0)

                # Lưu dữ liệu
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                row.extend([Video_name])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display hip angle
                cv2.putText(image, "hip_angle", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(hip_angle, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_ankle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Hiển thị ảnh
            # if(hip_angle/shoulder_hip_ankle_angle <0.52):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

In [ ]:
import os

# df = pd.DataFrame(columns=header)
# df_test = pd.DataFrame(columns=header)
# df_va = pd.DataFrame(columns=header)

#Path
# video_folder = "Data/train/Xuong_qua_xau"
video_folder = "Data/Train/Gap_lung"
# video_folder = "Data/train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "train")

video_folder = "Data/Test/Gap_lung"
# video_folder = "Data/test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "test")

video_folder = "Data/Validation/Gap_lung"
# video_folder = "Data/validate/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_5_error(video_folder, video_file, "validate")

In [ ]:
df

# 6. Thêm data đúng

In [ ]:
def detection_correct(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                landmarks = results.pose_landmarks.landmark

                #Kiểm tra lỗi
                row = []

                row.append(0)
                for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
                
                row.extend([Video_name])
                
                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")


            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Hiển thị ảnh
            # if(hip_angle/shoulder_hip_ankle_angle <0.52):
            #     plt.imshow(image)
            #     plt.axis("off")  # Ẩn trục tọa độ
            #     plt.show()

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()


In [ ]:
import os

#Path
video_folder = "Data/Train/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "train")

video_folder = "Data/Test/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "test")

video_folder = "Data/Validation/Correct"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_correct(video_folder, video_file, "validate")


In [ ]:
df

# 7. Đưa dataframe ra csv

In [ ]:
def save_splits_to_csv(train_df, test_df, val_df, output_dir="dataset"):
    """
    Lưu các DataFrame train, test, validation ra file CSV

    Args:
        train_df (pd.DataFrame): Dữ liệu train
        test_df (pd.DataFrame): Dữ liệu test
        val_df (pd.DataFrame): Dữ liệu validation
        output_dir (str): Thư mục lưu file CSV (mặc định là 'output')
    """
    import os
    os.makedirs(output_dir, exist_ok=True)  # Tạo thư mục nếu chưa có
    
    train_df.to_csv(f"{output_dir}/train.csv", index=False)
    test_df.to_csv(f"{output_dir}/test.csv", index=False)
    val_df.to_csv(f"{output_dir}/validation.csv", index=False)
    
    print(f"Dữ liệu đã được lưu vào thư mục '{output_dir}'.")

In [ ]:
save_splits_to_csv(df, df_test, df_va, "dataset")